In [3]:
import os
from pathlib import Path

In [5]:
%%bash

hashFrag -h

usage: hashFrag [-h]
                {blastn_module,filter_candidates_module,filter_test_split_module,stratify_test_split_module,identify_homologous_groups_module,create_orthogonal_splits_module,filter_existing_splits,stratify_test_split,create_orthogonal_splits}
                ...

hashFrag is a tool developed to mitigate the impacts of homology-based data leakage
in sequence-to-expression models. By identifying homology (based on pairwise
alignment scores) in a sequence dataset, this tool can be used to filter homologous
sequences spanning existing train-test splits (e.g., chromosomal splits), stratify
a test split according to different levels of homology, or create homology-aware
train-test splits.

positional arguments:
  {blastn_module,filter_candidates_module,filter_test_split_module,stratify_test_split_module,identify_homologous_groups_module,create_orthogonal_splits_module,filter_existing_splits,stratify_test_split,create_orthogonal_splits}
    blastn_module       A wrapper s

# Introduction

> This notebook refers to the case when users have a nucleotide sequence dataset and are interested in creating homology-aware train-test data splits for sequence-to-expression models.

The basic workflow is provided with respect a subsampled MPRA dataset (K562): a 10,000-sequence FASTA file (provided in the `data` directory).

hashFrag is a command line tool. This notebook serves as a resource to provide example calls and explanations to each step in the process.


# Section 0: Setup

In [72]:
data_dir   = "/home/brett/work/OrthogonalTrainValSplits/hashFrag/data"
fasta_path = os.path.join(data_dir,"K562.sample_10000.fa.gz")
label      = os.path.basename(fasta_path).replace(".fa.gz","")
score_path = os.path.join(data_dir,f"{label}.pairwise_scores.csv.gz")
work_dir   = os.path.join(data_dir,f"{label}.hashFrag.create_splits.work")
Path(work_dir).mkdir(parents=True,exist_ok=True)
print(work_dir)

blast_dir = os.path.join(work_dir,"blast_partitions")
Path(blast_dir).mkdir(parents=True,exist_ok=True)

/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work


# tl;dr

In [8]:
%%bash

FASTA_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.fa.gz
WORK_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work

WORD_SIZE=7
MAX_TARGET_SEQS=8000 # size of train dataset
E_VALUE=100
DUST=no
THRESHOLD=60
SEED=21
P_TRAIN=0.8
P_TEST=0.2
N_SPLITS=10

hashFrag create_orthogonal_splits \
--fasta_path $FASTA_PATH \
--word_size $WORD_SIZE \
--max_target_seqs $MAX_TARGET_SEQS \
--e_value $E_VALUE \
--dust $DUST \
--threshold $THRESHOLD \
--seed $SEED \
--p_train $P_TRAIN \
--p_test $P_TEST \
--n_splits $N_SPLITS \
--output_dir $WORK_DIR

2024-12-29 14:36:00 - numexpr.utils - INFO - Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2024-12-29 14:36:00 - numexpr.utils - INFO - Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
2024-12-29 14:36:00 - numexpr.utils - INFO - NumExpr defaulting to 16 threads.
2024-12-29 14:36:14 - matplotlib - DEBUG - matplotlib data path: /home/brett/envs/miniconda3/envs/py39R41/lib/python3.9/site-packages/matplotlib/mpl-data
2024-12-29 14:36:15 - matplotlib - DEBUG - CONFIGDIR=/home/brett/.config/matplotlib
2024-12-29 14:36:15 - matplotlib - DEBUG - interactive is False
2024-12-29 14:36:15 - matplotlib - DEBUG - platform is linux
2024-12-29 14:36:16 - matplotlib - DEBUG - CACHEDIR=/home/brett/.cache/matplotlib
2024-12-29 14:36:16 - matplotlib.font_manager - DEBUG - Using fontManager instance from /home/brett/.cache/matplotlib/fontlist-v330.json
2024-12-29 14:37:02 - pipeline - 

# Section 1 - Identifying candidate similar sequences

The process of identifying candidate pairs of similar sequences involves first creating a BLAST database of the dataset, and then querying each sequence against the database. The BLASTn algorithm returns pairwise matches that represent potential cases of homology. 

Successful identification of cases of homology is paramount to effectively mitigate homology-based data leakage. As such, we configure the BLASTn parameters such that recall is maximized, even if it comes at the expense of increased false-positives. Here we consider the following parameters of BLASTn:

* word_size: smaller word sizes results in more exact word matches found between the query and sequences in the database, leading to more alignment score calculations being initialized.
* max_target_seqs: set to the size of the database to remove any constraints and allow for all possible candidate sequences to be returned for a given query.
* evalue: the e-value statistic is a measure of how likely you observe the alignment by chance (lower value corresponds to less likely to observe). By increasing the e-value threshold, less stringent matches that could be due to chance are returned.
* dust: by setting dust off, low-complexity (e.g., repetitive sequences) are no longer masked/filtered out.

In [9]:
%%bash

FASTA_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.fa.gz
WORK_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work

WORD_SIZE=7
MAX_TARGET_SEQS=10000 # size of dataset
E_VALUE=100
DUST=no

hashFrag blastn_module \
-f $FASTA_PATH \
-m $MAX_TARGET_SEQS \
-w $WORD_SIZE \
-e $E_VALUE \
-d $DUST \
--blastdb_label "hashFrag" \
-o $WORK_DIR

2024-12-29 14:45:33 - blastn_module - INFO - Calling module...
2024-12-29 14:45:33 - blastn_module - INFO - One FASTA files detected. Computing pairwise BLAST comparisons for all sequence-pairs...
2024-12-29 14:45:33 - blastn_module - INFO - Existing BLAST database found. Path: /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work/hashFrag.blastdb
	skipping `makeblastdb` call.
2024-12-29 14:45:33 - blastn_module - INFO - Existing BLAST results file found. Path: /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work/hashFrag.blastn.out
	skipping `blastn` call.
2024-12-29 14:45:33 - blastn_module - INFO - Module execution completed.



# Section 2: Filter false-positives based on a defined threshold

The next step involves filtering candidate pairings with alignment scores lower than the specified threshold. There are two different modes of hashFrag depending on what alignment score is selected.

* `hashFrag-lightning` is the faster version where the alignment score computed from the BLAST output file. BLASTn is a heuristic method and the alignment scores were found to highly correlate with the optimal alignment scores; however, its underestimation of homology in some cases can lead to slightly worse recall. 
* `hashFrag-pure` is the slower but more comprehensive method that is based on the optimal, Smith-Waterman local alignment scores between pairs of sequences. The calculation of optimal alignment scores incurs an additional cost to filtering.

An alignment score threshold of 60 was determined to be appropriate based on an analysis looking at alignment scores between dinucleotide shuffled (i.e., random) sequences.

## Section 2.1: Lightning mode

In [10]:
%%bash

WORK_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work
INPUT_PATH=$WORK_DIR/hashFrag.blastn.out
MODE=lightning
THRESHOLD=60

hashFrag filter_candidates_module -m $MODE -i $INPUT_PATH -t $THRESHOLD -o $WORK_DIR

2024-12-29 14:46:00 - numexpr.utils - INFO - Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2024-12-29 14:46:00 - numexpr.utils - INFO - Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
2024-12-29 14:46:00 - numexpr.utils - INFO - NumExpr defaulting to 16 threads.
2024-12-29 14:46:08 - filter_candidates_module - INFO - Calling module...
2024-12-29 14:46:08 - filter_candidates_module - INFO - Filtering based on corrected BLAST alignment scores (lightning mode).
2024-12-29 14:46:09 - filter_candidates_module - INFO - Filtered results written to: /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work/hashFrag_lightning.similar_pairs.tsv.gz
2024-12-29 14:46:09 - filter_candidates_module - INFO - Module execution completed.



## Section 2.2: Pure mode (optional)

To limit memory usage, we'll start by partitioning the blast output file based on size. 

In [11]:
%%bash

WORK_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work
BLAST_PATH=$WORK_DIR/hashFrag.blastn.out
BLAST_DIR=$WORK_DIR/blast_partitions

LABEL=$( basename -s ".out" $BLAST_PATH )

mkdir -p $BLAST_DIR
cd $BLAST_DIR

SPLIT_SIZE=100000

split -l $SPLIT_SIZE -a 4 --additional-suffix=.tsv $BLAST_PATH ${LABEL}.partition_

This bash script execution will call a custom python script that computes pairwise Smith-Waterman local alignment scores for the candidate pairs of sequences identified by the BLASTn algorithm. Note that this could feasibly be replaced with any scoring metric of interest.

The expected format output files consists of a tab-delimited file with 3 columns: the query sequence iD, the target seqeuence ID, and their alignment score:
```
seq1    seq2    100
seq3    seq4    30
seq5    seq6    65
```

In [12]:
%%bash

cd ../src/external

FASTA_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.fa.gz
WORK_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work
BLAST_DIR=$WORK_DIR/blast_partitions

for PARTITIONED_BLAST_PATH in $BLAST_DIR/*.blastn.partition_*.tsv
do
    echo $PARTITIONED_BLAST_PATH
    bash compute_blast_candidate_SW_scores.sh $FASTA_PATH $PARTITIONED_BLAST_PATH
done

/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work/blast_partitions/hashFrag.blastn.partition_aaaa.tsv
/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work/blast_partitions/hashFrag.blastn.partition_aaab.tsv
/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work/blast_partitions/hashFrag.blastn.partition_aaac.tsv
/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work/blast_partitions/hashFrag.blastn.partition_aaad.tsv
/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work/blast_partitions/hashFrag.blastn.partition_aaae.tsv
/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work/blast_partitions/hashFrag.blastn.partition_aaaf.tsv
/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.ha

The SW scores for candidate pairs of sequences can subsequently be concatenated into a single `.tsv` file.

In [13]:
%%bash

WORK_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work
BLAST_DIR=$WORK_DIR/blast_partitions

zcat $BLAST_DIR/*.pairwise_scores.tsv.gz | gzip > $WORK_DIR/K562.sample_10000.blastn_candidates.custom_scores.tsv.gz

Rather than using the heuristic alignment scores provided by the BLASTn algorithm, we can filter false-positives based on SW alignment scores. Make sure to set the mode to `pure`.

In [14]:
%%bash

WORK_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work
INPUT_PATH=$WORK_DIR/K562.sample_10000.blastn_candidates.custom_scores.tsv.gz
MODE=pure
THRESHOLD=60

hashFrag filter_candidates_module -m $MODE -i $INPUT_PATH -t $THRESHOLD -o $WORK_DIR

2024-12-29 14:53:06 - numexpr.utils - INFO - Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2024-12-29 14:53:06 - numexpr.utils - INFO - Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
2024-12-29 14:53:06 - numexpr.utils - INFO - NumExpr defaulting to 16 threads.
2024-12-29 14:53:06 - filter_candidates_module - INFO - Calling module...
2024-12-29 14:53:06 - filter_candidates_module - INFO - Filtering based on precomputed alignment scores (pure mode).
2024-12-29 14:53:07 - filter_candidates_module - INFO - Filtered results written to: /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work/hashFrag_pure.similar_pairs.tsv.gz
2024-12-29 14:53:07 - filter_candidates_module - INFO - Module execution completed.



# Section 3: Determine groups of homology

There are often distinct groups of sequences exhibiting different cases of homology throughout the dataset. To determine such groups, we represent the "hits" (i.e., pairs of sequences with an alignment score greater than the threshold) as a sparse adjacency matrix. A graph can then be constructed, where nodes correspond to sequences and edges denote shared homology between the two sequences. The process of identifying groups of homology can readily be solved by identifying disconnected subgraphs. 

An efficient implementation for this graph-based task is provided in the `igraph` Python library.

## Section 3.1: `lightning`-filtered homologous pairs

In [15]:
%%bash

WORK_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work
HITS_PATH=$WORK_DIR/hashFrag_lightning.similar_pairs.tsv.gz
OUTPUT_PATH=$WORK_DIR/homologous_groups.lightning.csv

hashFrag identify_homologous_groups_module -i $HITS_PATH -o $OUTPUT_PATH

2024-12-29 14:53:18 - matplotlib - DEBUG - matplotlib data path: /home/brett/envs/miniconda3/envs/py39R41/lib/python3.9/site-packages/matplotlib/mpl-data
2024-12-29 14:53:18 - matplotlib - DEBUG - CONFIGDIR=/home/brett/.config/matplotlib
2024-12-29 14:53:18 - matplotlib - DEBUG - interactive is False
2024-12-29 14:53:18 - matplotlib - DEBUG - platform is linux
2024-12-29 14:53:19 - matplotlib - DEBUG - CACHEDIR=/home/brett/.cache/matplotlib
2024-12-29 14:53:19 - matplotlib.font_manager - DEBUG - Using fontManager instance from /home/brett/.cache/matplotlib/fontlist-v330.json
2024-12-29 14:53:38 - numexpr.utils - INFO - Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2024-12-29 14:53:38 - numexpr.utils - INFO - Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
2024-12-29 14:53:38 - numexpr.utils - INFO - NumExpr defaulting to 16 threads.
2024-12-29 14:53:54 - identify_ho

## Section 3.2: `pure`-filtered homologous pairs

In [16]:
%%bash

WORK_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work
HITS_PATH=$WORK_DIR/hashFrag_pure.similar_pairs.tsv.gz
OUTPUT_PATH=$WORK_DIR/homologous_groups.pure.csv

hashFrag identify_homologous_groups_module -i $HITS_PATH -o $OUTPUT_PATH

2024-12-29 14:53:55 - matplotlib - DEBUG - matplotlib data path: /home/brett/envs/miniconda3/envs/py39R41/lib/python3.9/site-packages/matplotlib/mpl-data
2024-12-29 14:53:55 - matplotlib - DEBUG - CONFIGDIR=/home/brett/.config/matplotlib
2024-12-29 14:53:55 - matplotlib - DEBUG - interactive is False
2024-12-29 14:53:55 - matplotlib - DEBUG - platform is linux
2024-12-29 14:53:55 - matplotlib - DEBUG - CACHEDIR=/home/brett/.cache/matplotlib
2024-12-29 14:53:55 - matplotlib.font_manager - DEBUG - Using fontManager instance from /home/brett/.cache/matplotlib/fontlist-v330.json
2024-12-29 14:53:55 - numexpr.utils - INFO - Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2024-12-29 14:53:55 - numexpr.utils - INFO - Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
2024-12-29 14:53:55 - numexpr.utils - INFO - NumExpr defaulting to 16 threads.
2024-12-29 14:53:56 - identify_ho

# Section 4: Use case(s)

Upon identifying groups of sequences exhibiting high similarity (i.e., homology), we can create train-test data splits using a graph-based method. Specifically, by representing sequences as nodes and using edges to denote whether sequences were found to be homologous (yes or no), identifying homologous groups of sequences can be reduced to the task of identifying all disconnected subgraphs in the population. 

## Creating homology-aware data splits

Below we show how splits can be created based on the homologous groups identified from either the `hashFrag-lightning` or `hashFrag-pure` methods.

In [17]:
%%bash

FASTA_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.fa.gz
WORK_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work
HOMOLOGY_PATH=$WORK_DIR/homologous_groups.lightning.csv # lightning mode
OUT_DIR=$WORK_DIR

hashFrag create_orthogonal_splits_module -f $FASTA_PATH -i $HOMOLOGY_PATH -n 10 -o $OUT_DIR

2024-12-29 14:53:56 - create_orthogonal_splits_module - INFO - Calling module...
2024-12-29 14:53:56 - create_orthogonal_splits_module - INFO - Creating 10 orthogonal splits in directory: /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work
2024-12-29 14:53:59 - create_orthogonal_splits_module - INFO - Module execution completed.



In [18]:
%%bash

FASTA_PATH=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.fa.gz
WORK_DIR=/home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work
HOMOLOGY_PATH=$WORK_DIR/homologous_groups.pure.csv # pure mode
OUT_DIR=$WORK_DIR

hashFrag create_orthogonal_splits_module -f $FASTA_PATH -i $HOMOLOGY_PATH -n 10 -o $OUT_DIR

2024-12-29 14:54:00 - create_orthogonal_splits_module - INFO - Calling module...
2024-12-29 14:54:00 - create_orthogonal_splits_module - INFO - Creating 10 orthogonal splits in directory: /home/brett/work/OrthogonalTrainValSplits/hashFrag/data/K562.sample_10000.hashFrag.create_splits.work
2024-12-29 14:54:02 - create_orthogonal_splits_module - INFO - Module execution completed.

